# 1. Imports

In [1]:
from src.configuration import Configuration
from src.fetch_data import DataFetcher
from aggregate_data import DataAggregator
from src.dim import Dims
from src.facts import Facts
from src.dicts import Dicts
import pandas as pd
import time

# 2. Variables

In [2]:
config = Configuration('src/config.cfg')
fetcher = DataFetcher(config)
DATAAGGREGATOR = DataAggregator(config, fetcher)
DIMS = Dims(config, fetcher)
DICTS = Dicts(config, fetcher)
FACTS = Facts(config, fetcher)

pd.set_option('display.max_columns', None)

25/11/03 20:31:30 WARN Utils: Your hostname, MacBook-Pro-Grzegorz.local resolves to a loopback address: 127.0.0.1; using 192.168.0.3 instead (on interface en0)
25/11/03 20:31:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/03 20:31:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# 3. Data

## 3.1 Driver related data

In [3]:
driver_last_5_races_result = DATAAGGREGATOR.get_last_races_result(5,"Race", "position")
time.sleep(5)
driver_last_5_quali_result = DATAAGGREGATOR.get_last_races_result(5,"Qualifying", "position")
time.sleep(5)
driver_last_5_avg_race_position = DATAAGGREGATOR.get_last_races_result(5, "Race", "avg")
time.sleep(5)
driver_last_5_std_race_position = DATAAGGREGATOR.get_last_races_result(5, "Race", "std")
time.sleep(5)
driver_last_5_avg_quali_position = DATAAGGREGATOR.get_last_races_result(5, "Qualifying", "avg")
time.sleep(5)
driver_last_5_std_quali_position = DATAAGGREGATOR.get_last_races_result(5, "Qualifying", "std")
time.sleep(5)
driver_points_gathered = DATAAGGREGATOR.get_racer_team_points("driver")
time.sleep(10)
driver_gap_to_teammate = DATAAGGREGATOR.calculate_gap_to_teammate()
time.sleep(5)
driver_gap_to_leader = DATAAGGREGATOR.calculate_gap_to_leader()
dict_world_champions = DICTS.dict_world_champions()
time.sleep(10)
pit_stops_efficiency = DATAAGGREGATOR.calculate_pit_stop_efficiency()

/Users/grzegorznaporowski/Desktop/Portfolio/F1/aggregate_data.py:159: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_filtered["points_gap_to_teammate"] = merged_filtered["points_gained"] - merged_filtered["points_gained_other"]


## 3.2 Team related data

In [7]:
team_points_gathered = DATAAGGREGATOR.get_racer_team_points("team")
time.sleep(5)
team_gap_to_leader = DATAAGGREGATOR.calculate_gap_to_best_team()

## 3.3 Session Related Data

In [8]:
dim_session = DIMS.dim_sessions()
dim_session_race = dim_session[dim_session["session_name"]=="Race"]
needed_cols = ["key", "location", "year", "is_current_season","country_name", "date_start", "session_key"]
dim_session_race = dim_session_race[needed_cols]
race_sequence = DATAAGGREGATOR.calculate_race_sequence_number()

In [5]:
needed_cols = ["driver_number","key","number_of_laps","dnf","dns","dsq","duration"]
fact_session_result = FACTS.fact_session_results()
race_duration = fact_session_result.merge(
    dim_session_race,
    on = "session_key",
    how="inner"
)

race_duration = race_duration[needed_cols]
race_duration["duration"] = race_duration["duration"].fillna(9999)

/var/folders/rd/z4g97_m5135b3yksvpxnxw1m0000gn/T/ipykernel_5169/1971916267.py:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  race_duration["duration"] = race_duration["duration"].fillna(9999)


## 3.4 Weather Data

In [10]:
dim_session_quali = dim_session[dim_session["session_name"]=="Qualifying"]
needed_cols = ["key", "location", "year", "is_current_season","country_name", "date_start", "session_key"]
dim_session_quali = dim_session_quali[needed_cols]
dim_weather_race = DIMS.dim_weather(race_type="Race")
time.sleep(20)
dim_weather_quali = DIMS.dim_weather(race_type="Qualifying")

### 3.4.1 Race Weather

In [42]:
race_weather = dim_weather_race.merge(
    dim_session_race,
    on="session_key",
    how="inner"
)
cols_race = [col for col in race_weather.columns if "race" in col or col == "key"]
race_weather = race_weather[cols_race]

### 3.5 Qualification Weather

In [44]:
quali_weather = dim_weather_quali.merge(
    dim_session_quali,
    on="session_key",
    how="inner"
)
cols_quali = [col for col in quali_weather.columns if "quali" in col or col == "key"]
quali_weather = quali_weather[cols_quali]
quali_weather

,wind_direction_qualifying_avg,wind_speed_qualifying_avg,has_rainfall_qualifying,track_temperature_qualifying_avg,air_temperature_qualifying_avg,humidity_qualifying_avg,pressure_qualifying_avg,key
0,213.789474,0.607368,0,28.085263,23.880000,21.652632,1017.641053,Sakhir2023
1,276.762500,1.547500,0,31.666250,27.023750,44.737500,1010.480000,Jeddah2023
2,127.189474,1.322105,1,22.712632,15.204211,53.400000,1018.930526,Melbourne2023
3,175.349057,0.795283,0,33.085849,19.461321,80.622642,1011.652830,Baku2023
4,140.655172,3.437931,0,43.266667,28.644828,59.206897,1017.397701,Miami2023
...,...,...,...,...,...,...,...,...
61,242.064935,2.300000,0,38.670130,26.132468,13.818182,1015.151948,Shanghai2025
62,139.058824,2.984706,0,32.141176,14.695294,58.623529,1013.291765,Suzuka2025
63,22.344828,3.132184,0,30.894253,26.296552,57.114943,1004.088506,Sakhir2025
64,221.662791,0.856977,0,37.225581,29.575581,57.569767,1006.300000,Jeddah2025


# 3.5 Joined Data

In [9]:
df = (
    dim_session
    .merge(driver_last_5_races_result, on="key", how="inner")
    .merge(driver_last_5_quali_result, on=["key", "driver_number"], how="inner", suffixes=["_race", "_quali"])
    .merge(driver_last_5_avg_race_position, on=["driver_number", "key"])
    .merge(driver_last_5_std_race_position, on=["driver_number", "key"])
    .merge(driver_last_5_avg_quali_position, on=["driver_number", "key"])
    .merge(driver_last_5_std_quali_position, on=["driver_number", "key"])
    .merge(driver_points_gathered[["driver_number", "key", "points_gained"]], on=["driver_number", "key"])
    .merge(driver_gap_to_teammate, on=["driver_number", "key"])
    .merge(driver_gap_to_leader, on=["driver_number", "key"])
    .merge(team_points_gathered, on=["driver_number", "key"])
    .merge(team_gap_to_leader, on=["driver_number", "key"])
    .merge(dict_world_champions,on="driver_number",how="left")
    .fillna({"titles_count": 0})
    .assign(world_champion=lambda x: x["titles_count"].astype(int))
    .merge(race_duration, on=["driver_number","key"])
    .merge(pit_stops_efficiency, on=["driver_number","key"])
    # .merge(race_weather, on="key") # Long loading time - off for now 
    # .merge(quali_weather, on="key") # Long loading time - off for now 
    .merge(race_sequence, on="key")
)


In [10]:
df.columns

Index(['session_key', 'location', 'date_start', 'date_end', 'session_name',
       'country_code', 'country_name', 'year', 'is_current_season', 'key',
       'driver_number', 'position_race', 'last_race_pos_1', 'last_race_pos_2',
       'last_race_pos_3', 'last_race_pos_4', 'last_race_pos_5',
       'position_quali', 'last_qualifying_pos_1', 'last_qualifying_pos_2',
       'last_qualifying_pos_3', 'last_qualifying_pos_4',
       'last_qualifying_pos_5', 'avg_last_5_race', 'std_last_5_race',
       'avg_last_5_qualifying', 'std_last_5_qualifying', 'points_gained',
       'points_gap_to_teammate', 'gap_to_leader', 'team_name',
       'team_points_gained', 'gap_to_best_team', 'titles_count',
       'world_champion', 'number_of_laps', 'dnf', 'dns', 'dsq', 'duration',
       'pit_duration_avg', 'pit_stop_count', 'race_number'],
      dtype='object')